In [1]:
# Install required packages
!pip install --no-deps peft accelerate bitsandbytes
!pip install py7zr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 73.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.7/413.7 kB 36.0 MB/s eta 0:00:00


In [2]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
# Install Flash Attention 2 for softcapping support
import torch
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install --no-deps packaging ninja einops "flash-attn>=2.6.3"

In [3]:

from unsloth import FastLanguageModel
import torch
max_seq_length = 512 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-v0.3-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.11.10: Fast Mistral patching. Transformers:4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/157 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/137k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [4]:
# Apply PEFT (Parameter Efficient Fine-Tuning) to the loaded model
model = FastLanguageModel.get_peft_model(
    model,
    r=8,  # Reduced LoRA rank for lower VRAM usage
    target_modules=[
        "q_proj", "v_proj", "gate_proj",
    ],  # Minimal modules for task-specific fine-tuning
    lora_alpha=16,  # Scaling factor for LoRA; unchanged
    lora_dropout=0,  # Small dropout for better generalization
    bias="none",  # No additional bias to reduce memory
    use_gradient_checkpointing="unsloth",  # Optimized gradient checkpointing
    random_state=3407,  # Ensure reproducibility
    use_rslora=False,  # Disabling Rank Stabilized LoRA (default)
    loftq_config=None,  # Disabling LoftQ (default)
)


Not an error, but Unsloth cannot patch MLP layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Not an error, but Unsloth cannot patch Attention layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Not an error, but Unsloth cannot patch O projection layer with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Unsloth 2024.11.10 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [5]:
from datasets import load_dataset


In [7]:
dataset = load_dataset("conll2003", split="train[:1000]")

README.md:   0%|          | 0.00/12.3k [00:00<?, ?B/s]

conll2003.py:   0%|          | 0.00/9.57k [00:00<?, ?B/s]

The repository for conll2003 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/conll2003.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [8]:
# Print basic dataset information
print(f"\n## Dataset Overview")
print(f"Number of examples: {len(dataset)}")
print(f"Features: {dataset.features}")

# Print NER tag meanings
ner_tags = {
    0: 'O',      # Outside of a named entity
    1: 'B-PER',  # Beginning of person name
    2: 'I-PER',  # Inside of person name
    3: 'B-ORG',  # Beginning of organization
    4: 'I-ORG',  # Inside of organization
    5: 'B-LOC',  # Beginning of location
    6: 'I-LOC',  # Inside of location
    7: 'B-MISC', # Beginning of miscellaneous
    8: 'I-MISC'  # Inside of miscellaneous
}

print("\n## NER Tags:")
for idx, tag in ner_tags.items():
    print(f"{idx}: {tag}")

# Print sample examples
print("\n## Sample Examples")
for i in range(2):
    example = dataset[i]
    print(f"\nExample {i+1}:")
    print("Tokens:", example['tokens'])
    print("NER tags:", [ner_tags[tag] for tag in example['ner_tags']])

    # Print aligned tokens and their NER tags
    print("\nAligned tokens and tags:")
    for token, tag in zip(example['tokens'], example['ner_tags']):
        if tag != 0:  # Only print tokens that are named entities
            print(f"{token}: {ner_tags[tag]}")


## Dataset Overview
Number of examples: 1000
Features: {'id': Value(dtype='string', id=None), 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'pos_tags': Sequence(feature=ClassLabel(names=['"', "''", '#', '$', '(', ')', ',', '.', ':', '``', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'NN|SYM', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB'], id=None), length=-1, id=None), 'chunk_tags': Sequence(feature=ClassLabel(names=['O', 'B-ADJP', 'I-ADJP', 'B-ADVP', 'I-ADVP', 'B-CONJP', 'I-CONJP', 'B-INTJ', 'I-INTJ', 'B-LST', 'I-LST', 'B-NP', 'I-NP', 'B-PP', 'I-PP', 'B-PRT', 'I-PRT', 'B-SBAR', 'I-SBAR', 'B-UCP', 'I-UCP', 'B-VP', 'I-VP'], id=None), length=-1, id=None), 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

In [9]:

def format_instruction(example):
    text = " ".join(example['tokens'])
    entities = [(token, tag) for token, tag in zip(example['tokens'], example['ner_tags']) if tag != 'O']
    return {
        "text": f"Extract named entities from this text and classify them as PER (person), ORG (organization), or LOC (location):\n\nText: {text}\n\nEntities: {entities}"
    }

In [10]:
# Format dataset
formatted_dataset = dataset.map(format_instruction)

# Print example of formatted data
print("\n## Training Format Example")
print(formatted_dataset[0]['text'])

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]


## Training Format Example
Extract named entities from this text and classify them as PER (person), ORG (organization), or LOC (location):

Text: EU rejects German call to boycott British lamb .

Entities: [('EU', 3), ('rejects', 0), ('German', 7), ('call', 0), ('to', 0), ('boycott', 0), ('British', 7), ('lamb', 0), ('.', 0)]


In [11]:

from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported


trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=formatted_dataset,
    max_seq_length=max_seq_length,
    dataset_num_proc=2,  # Use 2 processors for dataset preprocessing
    packing=False,  # Packing disabled; useful for variable-length sequences
    args=TrainingArguments(
        per_device_train_batch_size=1,  # Lower batch size to fit within 10GB
        gradient_accumulation_steps=8,  # Maintain effective batch size
        warmup_steps=5,
        max_steps=50,  # Reduced steps for faster completion
        learning_rate=2e-4,  # Learning rate; can be adjusted if needed
        fp16=not is_bfloat16_supported(),  # Enable FP16 if bfloat16 not supported
        bf16=is_bfloat16_supported(),  # Enable bfloat16 if supported
        logging_steps=5,  # Log every 5 steps
        optim="adamw_8bit",  # Optimizer for memory efficiency
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,  # For reproducibility
        output_dir="./NER",  # Directory for model checkpoints
        report_to="none",  # Disable external reporting (e.g., WandB)
    ),
)


Map (num_proc=2):   0%|          | 0/1000 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [12]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
4.51 GB of memory reserved.


In [13]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 8
\        /    Total batch size = 8 | Total steps = 50
 "-____-"     Number of trainable parameters = 8,126,464


Step,Training Loss
5,1.316200
10,0.662000
15,0.544100
20,0.480800
25,0.463200
30,0.457800
35,0.434300
40,0.440000
45,0.401800
50,0.437800


In [14]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

677.047 seconds used for training.
11.28 minutes used for training.
Peak reserved memory = 5.059 GB.
Peak reserved memory for training = 0.549 GB.
Peak reserved memory % of max memory = 34.303 %.
Peak reserved memory for training % of max memory = 3.723 %.


In [15]:
# Save the model
trainer.save_model("./NER_assistant_final")



In [16]:
# First prepare the model for inference
model = FastLanguageModel.for_inference(model)

In [25]:

def extract_entities(text):
    prompt = f"Extract named entities from this text and classify them as PER (person), ORG (organization), or LOC (location):\n\nText: {text}\n\nEntities:"

    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=512
    ).to("cuda")

    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        temperature=0.1,  # Lower temperature for more focused entity extraction
        do_sample=True,
        top_p=0.9,
        num_return_sequences=1
    )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text.split("Entities:")[-1]



In [26]:
# Test cases with different types of entities
test_texts = [
    """
    Apple CEO Tim Cook announced a new partnership with Microsoft's Satya Nadella.
    """,

    """
    The United Nations headquarters in New York City hosted a climate change summit .
    """,

    """
    Tesla's factory in Shanghai, China has started producing electric vehicles for
    the Asian market.
    """
]



In [27]:
print("## Testing NER Model with Custom Text")
for i, text in enumerate(test_texts, 1):
    print(f"\nTest Case {i}:")
    print("Input Text:")
    print("-" * 50)
    print(text.strip())
    print("\nExtracted Entities:")
    print("-" * 50)
    print(extract_entities(text))
    print("=" * 70)



## Testing NER Model with Custom Text

Test Case 1:
Input Text:
--------------------------------------------------
Apple CEO Tim Cook announced a new partnership with Microsoft's Satya Nadella.

Extracted Entities:
--------------------------------------------------
  [('Apple', 3), ('CEO', 0), ('Tim', 1), ('Cook', 2), ('announced', 0), ('a', 0), ('new', 0), ('partnership', 0), ('with',

Test Case 2:
Input Text:
--------------------------------------------------
The United Nations headquarters in New York City hosted a climate change summit .

Extracted Entities:
--------------------------------------------------
  [('The', 0), ('United', 3), ('Nations', 4), ('headquarters', 0), ('in', 0), ('New', 3), ('York', 5), ('City', 0), ('hosted', 0), ('

Test Case 3:
Input Text:
--------------------------------------------------
Tesla's factory in Shanghai, China has started producing electric vehicles for 
    the Asian market.

Extracted Entities:
----------------------------------------------

In [28]:
model.push_to_hub("AagamShah08/mistral_7B_NER") # Online saving
tokenizer.push_to_hub("AagamShah08/mistral_7B_NER") # Online saving

README.md:   0%|          | 0.00/592 [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/32.5M [00:00<?, ?B/s]

Saved model to https://huggingface.co/AagamShah08/mistral_7B_NER


  0%|          | 0/1 [00:00<?, ?it/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]